# Notfall Analyse 1

## Gründe

Nach Fertigstellen des Project Proposals kamen wir zu folgender Feststellung: Unsere bisherigen Daten erlauben uns keine aussagekräftigen Analysen eines Zusammenhangs zw. Verspätung und Auslastung. Grund hierfür ist, dass bisher keine Werte für die Auslastung im Datensatz vorhanden waren. Es hätte eine pauschale Annahme stattgefunden, dass die Auslastung vor Corona höher war als während Corona (Analyse Verkehrsbetriebe Nürnberg), doch diese Annahme reicht für eine Analyse (Wie können die öffentlichen Verkehrsmittel in Zürich während der Hauptverkehrszeit optimiert werden) nicht aus. Es fehlen tatsächliche Werte!

Zudem sollte bisher analysiert werden, wie die Situation während Berufsverkehrszeiten verbessert werden kann. Dies beruht ebenfalls auf einer Hypothese, dass während der Berufsverkehrszeiten höhere Verspätungen aufkommen. Es fehlt allerdings eine Analyse, ob dies tatsächlich der Fall ist. Zudem schränkt dies eine Analyse über mögliche andere Korrelationen ein.

## Was fehlt?

- Daten über die genaue Auslastung, welche mit den bisherigen Daten verknüpft werden können
- Analyse über Korrelationen

## Strategie

### Daten

Die Stadt Zürich stellt auch Daten über die Auslastung der einzelnen Fahrzeuge bereit. Diese müssen analysiert werden, ob sie mit den Daten für die Verspätungen verknüpft werden können.

### Analyse über Korrelationen

Gesetzt des Falles, dass die Daten genügen Auskunft bieten und miteinander verknüpft werden können, müssen verschiedene Blickwinkel analysiert werden, um Optimierungspotenziale feststellen zu können:
- Wie sieht die Verteilung an Verspätungen aus? Gibt es überhaupt Optimierungsbedarf? (Boxplot)
- Wie sieht die Verteilung an der Auslastung aus? Gibt es hier Optimierungsbedarf? (Boxplot)
- Allgemeine Analyse über Korrelationen der einzelnen Attribute (Korrelationsmatrix)
- Fokus auf zeitliche Komponente: Gibt es Uhrzeiten, zu denen die Verspätung regelmäßig höher ist? Uhrzeiten, zu denen die Auslastung regelmäßig höher ist?
- Fokus auf örtliche Komponente: Gibt es Stationen, an denen Verspätungen/Auslastungen erhöht auftreten?
- Fokus auf Linie: Gibt es Linien, die tendenziell mehr von Verspätungen/Auslastungen betroffen sind?

Je nachdem, welche Auswertungen diese Analysen bringen, kann nach Optimierungspotenzialen gesucht werden, z.B.:
- Ab Auslastung von X% nehmen Verspätungen immer exponential zu. Wie kann dies Verbessert werden? (Was sind die Gründe? Zeit, Ort?)
- Zeitliche Komponente: Was ist der Unterschied zu anderen Uhrzeiten?
- Örtliche Komponente: Was ist der Unterschied zu anderen Stationen?
- Linie: Was ist der Unterschied zu anderen Linien?

### Probleme

- Datensätze zu Verspätungen sind aktuell je geplantem Halt jedes einzelnen Fahrzeugs der VBZ. Dies muss aggregiert werden, um eine Analyse zu ermöglichen.
- Zeitliche Analysen sind sehr komplex, da es hier viele Zusammenhänge geben kann (z.B. sind die Unterschiede je nach einer Uhrzeit an einem Tag festzustellen? Wiederholt sich dieses Phänomen an den nächsten Tagen? Sind die Unterschiede je Woche betrachtet ähnlich? Gibt es eine Saisonalität?). Hier muss eine Eindämmung stattfinden, die dennoch eine Aussagekräftige Analyse erlaubt.
- Jede Linie besitzt zwei Richtungen. Müssen diese getrennt voneinander analysiert werden?
- Es könnte sinnvoll sein, die einzelnen Linien gemäß der Reihenfolge ihrer Stationen zu rekonstruieren. Wie lassen sich die einzelnen Linien dann miteinander vergleichen?

## Ansatz

- Zeitliche Aggregation: Datensätze werden vorerst je Stunde aggregiert. Somit kann eine komplette Woche (Montag bis Sonntag) von 0 bis 24 Uhr betrachtet werden. Um Saisonalität auszuschließen, aber dennoch eine Analyse über mehrere Wochen zu ermöglichen, wird nur ein Monat betrachetet (4 Wochen). Ein Vergleich Vor-Corona/Während-Corona fällt entfällt. Es werden möglichst aktuelle Daten betrachtet.
- Jede Linie wird gemäß ihrer Richtung ausgewertet. Eine Rekonstruktion gemäß der Stations-Reihenfolge wird vorerst nicht berücksichtigt. Auch wenn es interessant sein könnte, ob und wenn ja, wie sich die Auswirkungen der Linien im Verlauf ihrer Fahrt verhalten, so ist dies im Moment zu spezifisch und im Hinblick der Deadline nicht realistisch. Die Verspätungen können trotzdem je Station aggregiert werden, wodurch ein Vergleich mit der geografischen ÖPNV Karte genügend Auskünfte bieten dürfte.
- Die Verspätungen werden aktuell durch die Differenz zwischen der IST-Ankunftszeit und der SOLL-Ankunftszeit berechnet. Dies bedeutet, dass durch ein Verfrühtes Eintreffen eines Fahrzeugs Minuswerte vorkommen können. Da eine Verfrühung eine Verspätung nicht aufwiegen kann, müssen beide Fälle getrennt voneinander betrachtet werden. Es macht Sinn, diese Unterteilung in weiteren Spalten festzuhalten (nominal mit true oder false). Darüber hinaus kann man eine Verspätung/Verfrühung von ein paar Sekunden nicht als solche einordnen. Es muss ein passender Grenzwert gefunden werden, ab wann eine Ankunft als Verfrühung oder Verspätung gilt. Laut der Schweizer SBB gilt ein Zug als pünktlich, wenn er am Zielbahnhof mit weniger als drei Minuten Verspätung eintrifft (https://company.sbb.ch/de/ueber-die-sbb/verantwortung/die-sbb-und-ihre-kunden/puenktlichkeit.html). Diese Grenze wird für die betrachteten Fahrzeuge adaptiert.

Hieraus erscheint folgender Aufbau für das Ziel-Dataframe als sinnvoll:

Fahrzeugtyp | Linie | Richtung | Haltestelle | Tag | Stunde | Kapazität | Fahrgäste | Auslastung | Ist-Ankunft | Soll-Ankunft | Zeit-Differenz | Ist-Verfrühung | Ist-Verspätung

Hierfür:
- Fahrzeugtyp (nominal): Bus oder Tram
- Linie (nominal): offizielle Liniennummer z.B. xy
- Richtung (nominal): 1 oder 2
- Haltestelle (nominal): offizielle Haltestellenbezeichnung z.B. xy
- Tag (nominal oder ordinal): Tag des Ereignisses nach TT.MM.YYYY
- Stunde (nominal oder ordinal): 0 bis 24 Uhr
- Kapazität ():
- Fahrgäste ():
- Auslastung ():
- Ist-Ankunftszeit (discrete): Ist-Ankunftszeit in Sekunden nach Mitternacht
- Soll-Ankunftszeit (discrete): Soll-Ankunftszeit in Sekunden nach Mitternacht
- Zeit-Differenz (discrete): Errechnt durch Ist-Ankunftszeit minus Soll-Ankunftszeit (in Sekunden)
- Ist-Verfrühung (nominal): true wenn Zeit-Differenz kleiner als -180 (3 Minuten), sonst false
- Ist-Verspätung (nominal): true wenn Zeit-Differenz größer als 180 (3 Minuten), sonst false

## Umsetzung

In [1]:
import pandas as pd
import altair as alt

# reset display of max rows to enable displaying all columns of large dfs
pd.options.display.max_columns = None

In [3]:
df_reisende = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrgastzahlen_ogd/download/REISENDE.csv", delimiter=';')
df_reisende.head(10)

,Tagtyp_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,Distanz,Tage_DTV,Tage_DWV,Tage_SA,Tage_SO,Nachtnetz,Tage_SA_N,Tage_SO_N,ID_Abschnitt
0,14,9,11,88877,1,2,154,153.0,19:14:00,18,3.27778,0.22222,5.50000,399,175.27317,175.27317,0.0,0.0,0,0,0,15400153
1,14,9,11,88877,1,3,153,152.0,19:14:54,18,2.77778,1.05556,7.22222,412,175.27317,175.27317,0.0,0.0,0,0,0,15300152
2,14,9,11,88877,1,4,152,151.0,19:16:12,18,1.72222,0.22222,8.72222,383,175.27317,175.27317,0.0,0.0,0,0,0,15200151
3,14,9,11,88877,1,5,151,150.0,19:17:06,18,3.00000,0.00000,11.72222,420,175.27317,175.27317,0.0,0.0,0,0,0,15100150
4,14,9,11,88877,1,6,150,68.0,19:18:18,18,1.50000,0.38889,12.83333,294,175.27317,175.27317,0.0,0.0,0,0,0,15000068
5,14,9,11,88877,1,7,68,156.0,19:19:42,18,4.61111,2.05556,15.38889,258,175.27317,175.27317,0.0,0.0,0,0,0,6800156
6,14,9,11,88877,1,8,156,157.0,19:21:30,18,18.61111,5.00000,29.00000,514,175.27317,175.27317,0.0,0.0,0,0,0,15600157
7,14,9,11,88877,1,9,157,158.0,19:23:06,18,2.77778,2.38889,29.38889,306,175.27317,175.27317,0.0,0.0,0,0,0,15700158
8,14,9,11,88877,1,10,158,159.0,19:24:06,18,2.33333,4.05556,27.66667,500,175.27317,175.27317,0.0,0.0,0,0,0,15800159
9,14,9,11,88877,1,11,159,160.0,19:25:24,18,2.44444,6.55556,23.55556,454,175.27317,175.27317,0.0,0.0,0,0,0,15900160


## Ergebnis

Nach intensiver Betrachtung der Daten sind wir zu folgemdem Ergebnis gekommen: Die Daten über die Auslastung können leider nicht mit den bisherigen Daten verknüpft werden, da sie bereits zu stark aggregiert vorliegen. Im Internet lassen sich auch sonst keine weiteren Daten über die Auslastung finden. Es muss eine andere Lösung gefunden werden.